In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# 1) Data inladen (zelfde folder)
co2 = pd.read_csv('owid-co2-data.csv', usecols=['country','year','co2_per_capita'])
hale = pd.read_csv('C64284D_ALL_LATEST.csv')

# 2) Healthy life expectancy filter en renaming
hale = hale.loc[
    (hale['IND_NAME']=='Healthy life expectancy (at birth)') &
    (hale['DIM_SEX']=='TOTAL'),
    ['GEO_NAME_SHORT','DIM_TIME','AMOUNT_N']
].rename(columns={
    'GEO_NAME_SHORT':'country',
    'DIM_TIME':'year',
    'AMOUNT_N':'healthy_life_expectancy'
})

# 3) Merge en filter voor 2019
co2['year'] = co2['year'].astype(int)
hale['year'] = hale['year'].astype(int)
merged = pd.merge(co2, hale, on=['country','year'], how='inner')
df2019 = merged.query("year == 2019")

# 4) Data voor plot
x = df2019['co2_per_capita']
y = df2019['healthy_life_expectancy']

# 5) Bereken OLS-trendline
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
line_x = np.linspace(x.min(), x.max(), 100)
line_y = slope * line_x + intercept

# 6) Plotten met legenda voor punten én trendline
plt.figure(figsize=(8,6))
plt.scatter(x, y, alpha=0.7, edgecolor='k', label='Country data points')
plt.plot(line_x, line_y, color='red', linestyle='--',
         label=f'Trendline (R²={r_value**2:.2f})')

plt.xlabel('CO₂ per Capita (tons)')
plt.ylabel('Healthy Life Expectancy (years)')
plt.title('CO₂ per Capita vs Healthy Life Expectancy (2019)')
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(title='Legend')
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'owid-co2-data.csv'

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# 1) Data inladen en voorbereiden
co2 = pd.read_csv('owid-co2-data.csv', usecols=['country','year','co2_per_capita'])
hale = pd.read_csv('C64284D_ALL_LATEST.csv')

# Harmoniseren landnaam
co2['country'] = co2['country'].replace({'United States':'United States of America'})

# Filter Healthy life expectancy
hale = (
    hale.loc[
        (hale['IND_NAME']=='Healthy life expectancy (at birth)') &
        (hale['DIM_SEX']=='TOTAL'),
        ['GEO_NAME_SHORT','DIM_TIME','AMOUNT_N']
    ]
    .rename(columns={
        'GEO_NAME_SHORT':'country',
        'DIM_TIME':'year',
        'AMOUNT_N':'healthy_life_expectancy'
    })
)

# Jaar als int, merge en filter voor 2019 + geselecteerde landen
co2['year'] = co2['year'].astype(int)
hale['year'] = hale['year'].astype(int)
merged = pd.merge(co2, hale, on=['country','year'], how='inner')

countries = [
    'United States of America',
    'France',
    'India'
]

df2019 = (
    merged
    .query("year == 2019 and country in @countries")
    .set_index('country')
    .loc[countries]
    .reset_index()
)

# 2) X-waarden en barbreedte
x = np.arange(len(countries))
width = 0.4

# 3) Bouw de interactieve figuur
fig = go.Figure()

# CO₂ per Capita (linker y-as)
fig.add_trace(go.Bar(
    x=x - width/2,
    y=df2019['co2_per_capita'],
    name='CO₂ per Capita (tons)',
    marker_color='orange',
    yaxis='y1',
    hovertemplate='CO₂: %{y} ton<extra></extra>'
))

# Healthy Life Expectancy (rechter y-as)
fig.add_trace(go.Bar(
    x=x + width/2,
    y=df2019['healthy_life_expectancy'],
    name='Healthy Life Expectancy (years)',
    marker_color='blue',
    yaxis='y2',
    hovertemplate='Gezonde levensverwachting: %{y} jaar<extra></extra>'
))

# 4) Layout met dubbele y-as en styling gelijk aan matplotlib
fig.update_layout(
    title="Emissions vs Healthy Life Expectancy: U.S. vs France vs India (2019)",
    xaxis=dict(
        tickmode='array',
        tickvals=x,
        ticktext=countries,
        tickangle=15,
        title='Country'
    ),
    yaxis=dict(
        title=dict(
            text='CO₂ per Capita (tons)',
            font=dict(color='orange')
        ),
        tickfont=dict(color='orange')
    ),
    yaxis2=dict(
        title=dict(
            text='Healthy Life Expectancy (years)',
            font=dict(color='blue')
        ),
        tickfont=dict(color='blue'),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        x=0.5,
        xanchor='center'
    ),
    bargap=0.2,
    margin=dict(t=80, b=50, l=60, r=60)
)

# 5) Toon interactieve grafiek
fig.show()

# Optioneel: exporteer naar standalone HTML
# fig.write_html("emissions_health_2019.html", include_plotlyjs='cdn')

In [ ]:
# ── Cel: Interactieve Plot met Plotly ──
import pandas as pd
import plotly.graph_objects as go

# Data inladen en voorbereiden (zelfde stappen als voor matplotlib)
co2 = pd.read_csv('owid-co2-data.csv', usecols=['country','year','co2_per_capita'])
hale = pd.read_csv('C64284D_ALL_LATEST.csv')

# Filter Healthy life expectancy
hale = (
    hale.loc[
        (hale['IND_NAME']=='Healthy life expectancy (at birth)') &
        (hale['DIM_SEX']=='TOTAL'),
        ['GEO_NAME_SHORT','DIM_TIME','AMOUNT_N']
    ]
    .rename(columns={
        'GEO_NAME_SHORT':'country',
        'DIM_TIME':'year',
        'AMOUNT_N':'healthy_life_expectancy'
    })
)

# Year kolommen als int en merge voor China 2000–2021
co2['year'] = co2['year'].astype(int)
hale['year'] = hale['year'].astype(int)
co2_china = co2.query("country=='China' and 2000 <= year <= 2021")
hale_china = hale.query("country=='China' and 2000 <= year <= 2021")
china = pd.merge(co2_china, hale_china, on=['country','year']).sort_values('year')

# Maak de interactieve figuur
fig = go.Figure()

# CO₂ per capita (linker y-as)
fig.add_trace(go.Scatter(
    x=china['year'],
    y=china['co2_per_capita'],
    name='CO₂ per Capita (tons)',
    mode='lines+markers',
    marker=dict(color='orange'),
    line=dict(color='orange'),
    yaxis='y1'
))

# Healthy life expectancy (rechter y-as)
fig.add_trace(go.Scatter(
    x=china['year'],
    y=china['healthy_life_expectancy'],
    name='Healthy Life Expectancy (years)',
    mode='lines+markers',
    marker=dict(color='blue'),
    line=dict(color='blue'),
    yaxis='y2'
))

# Layout met dubbele y-as
fig.update_layout(
    title='China: Trend of CO₂ per Capita and Healthy Life Expectancy (2000–2021)',
    xaxis=dict(title='Year'),
    yaxis=dict(
        title=dict(
            text='CO₂ per Capita (tons)',
            font=dict(color='orange')
        ),
        tickfont=dict(color='orange')
    ),
    yaxis2=dict(
        title=dict(
            text='Healthy Life Expectancy (years)',
            font=dict(color='blue')
        ),
        tickfont=dict(color='blue'),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        orientation='h', 
        yanchor='bottom', 
        y=1.02, 
        x=0.5, 
        xanchor='center'
    )
)

# Toon interactieve grafiek
fig.show()

# Optioneel: exporteer naar standalone HTML (voor presentatie zonder code)
# fig.write_html("china_trend_interactive.html", include_plotlyjs='cdn')

In [ ]:
import pandas as pd
import plotly.express as px

# Load CO₂ data
co2 = pd.read_csv('owid-co2-data.csv', usecols=['iso_code', 'country', 'year', 'co2_per_capita', 'cumulative_co2'])

# Load and filter HALE data
hale = pd.read_csv('C64284D_ALL_LATEST.csv')
hale = (
    hale.loc[
        (hale['IND_NAME'] == 'Healthy life expectancy (at birth)') &
        (hale['DIM_SEX'] == 'TOTAL'),
        ['GEO_NAME_SHORT', 'DIM_TIME', 'AMOUNT_N']
    ]
    .rename(columns={
        'GEO_NAME_SHORT': 'country',
        'DIM_TIME': 'year',
        'AMOUNT_N': 'healthy_life_expectancy_at_birth'
    })
)

# Convert year columns to int
co2['year'] = co2['year'].astype(int)
hale['year'] = hale['year'].astype(int)

# Filter to specific years
years = [2000, 2010, 2015, 2019, 2021]
co2 = co2[co2['year'].isin(years)]
hale = hale[hale['year'].isin(years)]

# Filter valid country ISO codes
co2 = co2[co2['iso_code'].str.len() == 3]

# Merge datasets
merged_df = pd.merge(co2, hale, on=['country', 'year'], how='inner')
merged_df.dropna(inplace=True)

# Create scatter plot for 2019
fig = px.scatter(
    merged_df[merged_df['year'] == 2019],
    x='cumulative_co2',
    y='healthy_life_expectancy_at_birth',
    hover_name='country',
    log_x=True,
    labels={
        'cumulative_co2': 'Cumulative CO₂ Emissions (tonnes, log scale)',
        'healthy_life_expectancy_at_birth': 'Healthy Life Expectancy (years)'
    },
    title='Historical CO₂ Emissions vs Healthy Life Expectancy (2019)'
)

fig.show()



In [ ]:
import plotly.express as px

# World map with CO2 per capita and Healthy Life Expectancy
fig = px.scatter_geo(
    merged_df[merged_df['year'] == 2019],
    locations='iso_code',
    color='healthy_life_expectancy_at_birth',
    size='co2_per_capita',
    color_continuous_scale='Viridis',
    hover_name='country',
    title='Healthy Life Expectancy (color) and CO₂ Emissions per Capita (size) in 2019'
)

fig.show()


In [ ]:
import plotly.express as px

# Interactive scatter with animation over years
fig = px.scatter(
    merged_df, 
    x='co2_per_capita', 
    y='healthy_life_expectancy_at_birth',
    animation_frame='year', 
    animation_group='country',
    hover_name='country',
    range_x=[0, 35], 
    range_y=[40, 85],
    labels={
        'co2_per_capita': 'CO₂ per Capita (t/person)',
        'healthy_life_expectancy_at_birth': 'Healthy Life Expectancy (years)'
    },
    title='Emissions vs Healthy Life Expectancy (2000–2021)'
)

fig.show()
